In [33]:
import pandas as pd
import numpy as np

import missingno as msno

import statsmodels.api as sm
from scipy import stats

import matplotlib.pyplot as plt

from ucimlrepo import fetch_ucirepo 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [34]:
myocardial_infarction_complications = fetch_ucirepo(id=579) 
  
# Data
x = myocardial_infarction_complications.data.features
y = myocardial_infarction_complications.data.targets

In [35]:
#Classification task (T)
Y = pd.DataFrame(np.where(y[['LET_IS']] == 0, 1, 0),columns=['target'])

In [37]:
df = pd.concat([x,Y],axis=1)

In [38]:
#remove variables with too many missing values

df = df.loc[:, ~df.columns.isin(['IBS_NASL','KFK_BLOOD','NA_KB','D_AD_KBRIG','S_AD_KBRIG','LID_KB','NA_KB','NOT_NA_KB'])]

In [52]:
len(df.columns)

105

In [53]:
## Complete-case analysis reduces sample to n=544 observations. 

df_cc = df.copy().dropna()

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df_cc.loc[:, df_cc.columns != 'target'], df_cc['target'], test_size=0.3)

In [61]:
#Run logistic regression 

log_reg = LogisticRegression()

log_reg.fit(X_train, y_train)

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [56]:
log_reg.score(X_train, y_train)

0.9236842105263158

In [57]:
log_reg.score(X_test, y_test) # good performance but only on those 544

0.9024390243902439

In [67]:
#Evaluate performance on fresh outputs random values from $Y$ into the analysis
log_reg.score(df_cc.loc[:, df_cc.columns != 'target'].sample(200),Y.sample(n=200))

0.87

In [77]:
#Results from analysis

X2 = sm.add_constant(df_cc.loc[:, df_cc.columns != 'target'])
log_model = sm.OLS(df_cc['target'], X2)
log_model2 = log_model.fit()


In [78]:
print(log_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     2.368
Date:                Sat, 11 Nov 2023   Prob (F-statistic):           4.71e-09
Time:                        19:37:02   Log-Likelihood:                 4.9654
No. Observations:                 544   AIC:                             166.1
Df Residuals:                     456   BIC:                             544.4
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             1.3111      0.280      4.680

In [82]:
import statsmodels.api as sm

In [98]:
pvalues = log_model2.summary2().tables[1]['P>|t|'].to_frame()


In [107]:
#how many insignificant variables
insignificant = list(pvalues[pvalues['P>|t|'] >0.05].index)

In [108]:
df_cc = df_cc.loc[:, ~df_cc.columns.isin(insignificant)]

In [109]:
X_train, X_test, y_train, y_test = train_test_split(df_cc.loc[:, df_cc.columns != 'target'], df_cc['target'], test_size=0.3)

In [111]:
X2 = sm.add_constant(X_train)
log_model = sm.OLS(y_train, X2)
log_model2 = log_model.fit()

In [112]:
print(log_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     4.415
Date:                Sat, 11 Nov 2023   Prob (F-statistic):           2.59e-07
Time:                        19:47:19   Log-Likelihood:                -22.615
No. Observations:                 380   AIC:                             75.23
Df Residuals:                     365   BIC:                             134.3
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             1.1305      0.161      7.008

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
